In [0]:
#Importing libraries
import numpy as np
import pandas as pd

from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Activation
from keras.layers import Input
from keras.layers import Flatten
from keras.layers.recurrent import LSTM
from keras.layers.embeddings import Embedding
from keras.layers.merge import concatenate

from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [0]:
#Label Encoder to encode names of Terrorist Organisations
label_encoder = LabelEncoder()

In [0]:
#Scaling values for higher accuracy
scale_values  = StandardScaler()

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
df2 = pd.read_excel('/content/drive/My Drive/globalterrorismdb_0718dist.xlsx')

In [24]:
#Importing and reaading Global Terrorist Database using Pandas
Raw_Dataset = df2
Raw_Dataset.head()

,eventid,iyear,imonth,iday,approxdate,extended,resolution,country,country_txt,region,...,addnotes,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related
0,197000000001,1970,7,2,NaN,0,NaT,58,Dominican Republic,2,...,NaN,NaN,NaN,NaN,PGIS,0,0,0,0,NaN
1,197000000002,1970,0,0,NaN,0,NaT,130,Mexico,1,...,NaN,NaN,NaN,NaN,PGIS,0,1,1,1,NaN
2,197001000001,1970,1,0,NaN,0,NaT,160,Philippines,5,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
3,197001000002,1970,1,0,NaN,0,NaT,78,Greece,8,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
4,197001000003,1970,1,0,NaN,0,NaT,101,Japan,4,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN


In [0]:
#We will focus on those attacks only which were confirmed to be perpetrated by terrorist organisation
Dataset = Raw_Dataset[(Raw_Dataset.crit1 == 1) & 
                      (Raw_Dataset.crit2 == 1) & 
                      (Raw_Dataset.crit3 == 1) & 
                      (Raw_Dataset.doubtterr == 0)]

In [0]:
#Importing columns for dataset preparation
features = Dataset.loc[:,['extended',
                          'country',
                          'region',
                          'vicinity',
                          'attacktype1',
                          'targtype1',
                          'natlty1',
                          'weaptype1',
                          'weapsubtype1',
                          'summary',
                          'gname']]

In [0]:
#Dropping rows that are empty 
features = features.dropna() 

In [0]:
#Preparing target values and labeling
label_encoder.fit(features['gname'])
target = label_encoder.fit_transform(features['gname'])
#Y  = X.loc[:,['gname']]

In [0]:
#Here we tokenize words in summary to make it a feature
tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts(features['summary'])

x_seq = tokenizer.texts_to_sequences(features["summary"])

features_summary = sequence.pad_sequences(x_seq, maxlen=500, padding="post", value=0)

In [0]:
#We will now convert pandas dataframe to numpy arrays for processing 
features = features.values


In [31]:
features.shape

(88876, 11)

In [32]:
#values are scaled for more accuracy 
features_summary = scale_values.fit_transform(features_summary)
features[:,:-2] = scale_values.fit_transform(features[:,:-2])
Y = target.reshape(-1,1)
from sklearn.preprocessing import MinMaxScaler as scaler
Y = scale_values.fit_transform(Y)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int32 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int32 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by Sta

In [33]:
features_summary.shape

(88876, 500)

In [0]:
#Array with tokenize summary and array with other features are combined here to generate a single array
features_combine = np.concatenate((features[:,:-2],
                                   features_summary),
                                   axis = 1)

In [35]:
features_combine.shape

(88876, 509)

In [36]:
#spliting of array to train and test samples
x_train, x_test, y_train, y_test = train_test_split(features_combine, 
                                                    target, 
                                                    test_size=0.30, 
                                                    random_state=42)
#shapes are displayed of repective arrays
print (x_train.shape,y_train.shape)
print (x_test.shape,y_test.shape)

(62213, 509) (62213,)
(26663, 509) (26663,)


In [43]:

# first input model
visible1 = Input(shape=(9,))
hidden1 = Dense(320, activation='relu')(visible1)
hidden2 = Dense(640, activation='relu')(hidden1)
hidden3 = Dense(640, activation='relu')(hidden2)
output1 = Dense(1200, activation='relu')(hidden3)
# second input model
visible2 = Input(shape=(500,))
hidden4 = Dense(320, activation='relu')(visible2)
hidden5 = Dense(640, activation='relu')(hidden4)
hidden6 = Dense(640, activation='relu')(hidden5)
output2 = Dense(120, activation='relu')(hidden6)
# merge input models
merge = concatenate([output1, output2])
# interpretation model
hidden7 = Dense(320, activation='relu')(merge)
hidden8 = Dense(320, activation='relu')(hidden7)
hidden9 = Dense(320, activation='relu')(hidden8)
hidden10 = Dense(320, activation='relu')(hidden9)
hidden11 = Dense(320, activation='relu')(hidden10)
hidden12 = Dense(320, activation='relu')(hidden11)
output = Dense(len(np.unique(target)), activation='softmax')(hidden12)
model = Model(inputs=[visible1, visible2], outputs=output)
# summarize layers
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 9)            0                                            
__________________________________________________________________________________________________
input_7 (InputLayer)            (None, 500)          0                                            
__________________________________________________________________________________________________
dense_34 (Dense)                (None, 320)          3200        input_6[0][0]                    
__________________________________________________________________________________________________
dense_38 (Dense)                (None, 320)          160320      input_7[0][0]                    
__________________________________________________________________________________________________
dense_35 (

In [40]:
len(np.unique(target))

1473

In [0]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [46]:
model.fit([x_train[:,:9],
          x_train[:,9:]],
          y_train, epochs=100,
          
          
          
          validation_data=([x_test[:,:9],
                           x_test[:,9:]],
                           y_test),
                           verbose=2)

Train on 62213 samples, validate on 26663 samples
Epoch 1/100
 - 29s - loss: 1.5631 - acc: 0.6212 - val_loss: 1.4255 - val_acc: 0.6607
Epoch 2/100
 - 29s - loss: 1.2283 - acc: 0.6693 - val_loss: 1.2907 - val_acc: 0.6867
Epoch 3/100
 - 29s - loss: 1.1327 - acc: 0.6894 - val_loss: 1.2652 - val_acc: 0.6803
Epoch 4/100
 - 29s - loss: 1.0577 - acc: 0.7050 - val_loss: 1.3432 - val_acc: 0.6646
Epoch 5/100
 - 29s - loss: 1.0258 - acc: 0.7150 - val_loss: 1.2093 - val_acc: 0.7017
Epoch 6/100
 - 29s - loss: 0.9829 - acc: 0.7244 - val_loss: 1.2552 - val_acc: 0.7034
Epoch 7/100
 - 29s - loss: 0.9539 - acc: 0.7320 - val_loss: 1.2564 - val_acc: 0.6987
Epoch 8/100
 - 29s - loss: 0.9280 - acc: 0.7384 - val_loss: 1.2084 - val_acc: 0.7121
Epoch 9/100
 - 29s - loss: 0.9108 - acc: 0.7439 - val_loss: 1.1956 - val_acc: 0.7078
Epoch 10/100
 - 28s - loss: 0.8848 - acc: 0.7503 - val_loss: 1.2147 - val_acc: 0.7105
Epoch 11/100
 - 29s - loss: 0.8653 - acc: 0.7560 - val_loss: 1.2059 - val_acc: 0.7063
Epoch 12/100
